In [1]:
# ============================
# library
# ============================
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn import LayerNorm,TransformerEncoder
from torch.utils.data import DataLoader, TensorDataset, Dataset
import pickle
from tqdm import tqdm
import gc
import glob
from sklearn.preprocessing import StandardScaler,RobustScaler

In [2]:
# ============================
# constant
# ============================
SUB_PATH = "/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/sample_submission.csv"
DEFOG_DATA_PATH = "/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/defog/*.csv"
TDCSFOG_DATA_PATH = "/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/tdcsfog/*.csv"

In [3]:
# ============================
# settings
# ============================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bs = 32

In [4]:
# ============================
# settings
# ============================
tdcsfog_path1 = [f"/kaggle/input/fog-ex143/ex143_{i}.pth" for i in range(5)] # len 3000 cv TdcsfogRnnModel cv
tdcsfog_path3_1 = [f"/kaggle/input/fog-ex145/ex145_{i}.pth" for i in range(5)] # len 3000 TdcsfogRnnModel  
tdcsfog_path3_2 = [f"/kaggle/input/fog-ex146/ex146_{i}.pth" for i in range(5)] # len 3000 TdcsfogRnnModel 
tdcsfog_path3_3 = [f"/kaggle/input/fog-ex147/ex147_{i}.pth" for i in range(5)] # len 3000 TdcsfogRnnModel cv
tdcsfog_path4_1 = [f"/kaggle/input/fog-ex182/ex182_{i}.pth" for i in range(5)] # len 3000 TdcsfogRnnModel  
tdcsfog_path4_2 = [f"/kaggle/input/fog-ex183/ex183_{i}.pth" for i in range(5)] # len 3000 TdcsfogRnnModel 
tdcsfog_path4_3 = [f"/kaggle/input/fog-ex184/ex184_{i}.pth" for i in range(5)] # len 3000 TdcsfogRnnModel cv

defog_path2 = [f"/kaggle/input/fog-ex153/ex153_{i}.pth" for i in range(5)] # len 30000 defog1 
defog_path4 = [f"/kaggle/input/fog-ex179/ex179_{i}.pth" for i in range(5)] # len 30000 defog1
defog_path5 = [f"/kaggle/input/fog-ex185/ex185_{i}.pth" for i in range(5)] # len 30000 defog1
defog_path6 = [f"/kaggle/input/fog-ex204/ex204_{i}.pth" for i in range(5)] # len 30000 defog2
defog_path7 = [f"/kaggle/input/pd-exp238/fold{i}_best.pth" for i in [0, 1, 2, 3, 4]]  # len 30000 Defog3Model

In [5]:
# ============================
# Functions
# ============================

def preprocess(numerical_array, 
               mask_array,
               ):
    
    attention_mask = mask_array == 0

    return {
        'input_data_numerical_array': numerical_array,
        'input_data_mask_array': mask_array,
        'attention_mask': attention_mask,
    }

class FogDataset(Dataset):
    def __init__(self, numerical_array, 
                 mask_array,
                 train = True, y = None):
        self.numerical_array = numerical_array
        self.mask_array = mask_array
        self.train = train
        self.y = y
    
    def __len__(self):
        return len(self.numerical_array)

    def __getitem__(self, item):
        data = preprocess(
            self.numerical_array[item],
            self.mask_array[item],
            
        )

        # Return the processed data where the lists are converted to `torch.tensor`s
        if self.train : 
            return {
              'input_data_numerical_array': torch.tensor(data['input_data_numerical_array'],dtype=torch.float32),
              'input_data_mask_array':torch.tensor(data['input_data_mask_array'], dtype=torch.long),  
              'attention_mask': torch.tensor(data["attention_mask"], dtype=torch.bool),
              "y":torch.tensor(self.y[item], dtype=torch.float32)
               }
        else:
            return {
             'input_data_numerical_array': torch.tensor(data['input_data_numerical_array'],dtype=torch.float32),
              'input_data_mask_array':torch.tensor(data['input_data_mask_array'], dtype=torch.long),  
              'attention_mask': torch.tensor(data["attention_mask"], dtype=torch.bool),
               }
        




In [6]:
# ================================
# tdcsfog
# ================================
class TdcsfogRnnModel(nn.Module):
    def __init__(
        self, dropout=0.2,
        input_numerical_size=12,
        numeraical_linear_size = 64,
        model_size = 128,
        linear_out = 128,
        out_size=3):
        super(TdcsfogRnnModel, self).__init__()
        self.numerical_linear  = nn.Sequential(
                nn.Linear(input_numerical_size, numeraical_linear_size),
                nn.LayerNorm(numeraical_linear_size)
            )
        
        self.rnn = nn.GRU(numeraical_linear_size, model_size,
                            num_layers = 2, 
                            batch_first=True,
                            bidirectional=True)
        self.linear_out  = nn.Sequential(
                nn.Linear(model_size*2, 
                          linear_out),
                nn.LayerNorm(linear_out),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(linear_out, 
                          out_size))
        self._reinitialize()
        
    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
    
    def forward(self, numerical_array,
                mask_array,
                attention_mask):
        
        numerical_embedding = self.numerical_linear(numerical_array)
        output,_ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        return output
    
    
class TdcsfogRnnModel2(nn.Module):
    def __init__(
        self, dropout=0.2,
        input_numerical_size=12,
        numeraical_linear_size = 64,
        model_size = 128,
        linear_out = 128,
        out_size=3):
        super(TdcsfogRnnModel2, self).__init__()
        self.numerical_linear  = nn.Sequential(
                nn.Linear(input_numerical_size, numeraical_linear_size),
                nn.LayerNorm(numeraical_linear_size)
            )
        
        self.rnn = nn.GRU(numeraical_linear_size, model_size,
                            num_layers = 2, 
                            batch_first=True,
                            bidirectional=True)
        self.linear_out  = nn.Sequential(
                nn.Linear(model_size*2, 
                          linear_out),
                nn.LayerNorm(linear_out),
                nn.ReLU(),
                nn.Dropout(dropout))
        self.out1 = nn.Linear(linear_out, 
                          out_size)
        self.out2 = nn.Linear(linear_out, 
                          out_size)

        
        self._reinitialize()
        
    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
    
    def forward(self, numerical_array,
                mask_array,
                attention_mask):
        
        numerical_embedding = self.numerical_linear(numerical_array)
        output,_ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        output1 = self.out1(output)
        output2 = self.out2(output)
        return output1,output2

In [7]:
class DefogRnnModel(nn.Module):
    def __init__(
        self, dropout=0.2,
        input_numerical_size=9,
        numeraical_linear_size = 64,
        model_size = 128,
        linear_out = 128,
        out_size=3):
        super(DefogRnnModel, self).__init__()
        self.numerical_linear  = nn.Sequential(
                nn.Linear(input_numerical_size, numeraical_linear_size),
                nn.LayerNorm(numeraical_linear_size)
            )
        
        self.rnn = nn.GRU(numeraical_linear_size, model_size,
                            num_layers = 2, 
                            batch_first=True,
                            bidirectional=True)
        self.linear_out  = nn.Sequential(
                nn.Linear(model_size*2, 
                          linear_out),
                nn.LayerNorm(linear_out),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(linear_out, 
                          out_size))
        self._reinitialize()
        
    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
    
    def forward(self, numerical_array,
                mask_array,
                attention_mask):
        
        numerical_embedding = self.numerical_linear(numerical_array)
        output,_ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        return output
    
    
class DefogRnnModel2(nn.Module):
    def __init__(
        self, dropout=0.2,
        input_numerical_size=9,
        numeraical_linear_size = 96,
        model_size = 256,
        linear_out = 256,
        out_size=3):
        super(DefogRnnModel2, self).__init__()
        self.numerical_linear  = nn.Sequential(
                nn.Linear(input_numerical_size, numeraical_linear_size),
                nn.LayerNorm(numeraical_linear_size)
            )
        
        self.rnn = nn.GRU(numeraical_linear_size, model_size,
                            num_layers = 2, 
                            batch_first=True,
                            bidirectional=True)
        self.linear_out  = nn.Sequential(
                nn.Linear(model_size*2, 
                          linear_out),
                nn.LayerNorm(linear_out),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(linear_out, 
                          out_size))
        self._reinitialize()
        
    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
    
    def forward(self, numerical_array,
                mask_array,
                attention_mask):
        
        numerical_embedding = self.numerical_linear(numerical_array)
        output,_ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        return output
    
    
class Defog3Model(nn.Module):
    def __init__(
        self,
        dropout=0.2,
        input_numerical_size=9,
        numeraical_linear_size=64,
        model_size=128,
        linear_out=128,
        out_size=3,
    ):
        super(Defog3Model, self).__init__()
        self.numerical_linear = nn.Sequential(
                nn.Linear(input_numerical_size, numeraical_linear_size),
                nn.LayerNorm(numeraical_linear_size)
            )
        self.lstm = nn.GRU(
            numeraical_linear_size,
            model_size,
            num_layers=2,
            batch_first=True,
            bidirectional=True,
        )
        self.linear_out = nn.Sequential(
            nn.Linear(model_size*2, linear_out),
            nn.LayerNorm(linear_out),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(linear_out, out_size),
        )
        self._reinitialize()

    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'lstm' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)

    def forward(self, numerical_array, mask_array, attention_mask):
        numerical_embedding = self.numerical_linear(numerical_array)
        output, _ = self.lstm(numerical_embedding)
        output = self.linear_out(output)
        return output

In [8]:
def make_pred(test_loader,model):
    test_preds = []
    tk0 = tqdm(test_loader, total=len(test_loader))
    with torch.no_grad():  # Do not calculate gradient since we are only predicting
        # Predicting on validation set
        for d in tk0:
            input_data_numerical_array = d['input_data_numerical_array'].to(device)
            input_data_mask_array = d['input_data_mask_array'].to(device)
            attention_mask = d['attention_mask'].to(device)
            output = model(input_data_numerical_array, 
                       input_data_mask_array,
                       attention_mask)
            test_preds.append(output.sigmoid().cpu().numpy())
    test_preds = np.concatenate(test_preds,axis=0)
    return test_preds


def make_pred2(test_loader,model):
    test_preds = []
    tk0 = tqdm(test_loader, total=len(test_loader))
    with torch.no_grad():  # Do not calculate gradient since we are only predicting
        # Predicting on validation set
        for d in tk0:
            input_data_numerical_array = d['input_data_numerical_array'].to(device)
            input_data_mask_array = d['input_data_mask_array'].to(device)
            attention_mask = d['attention_mask'].to(device)
            output,_ = model(input_data_numerical_array, 
                       input_data_mask_array,
                       attention_mask)
            test_preds.append(output.sigmoid().cpu().numpy())
    test_preds = np.concatenate(test_preds,axis=0)
    return test_preds

In [9]:
# =======================
# main
# =======================
sub = pd.read_csv(SUB_PATH)

In [10]:
# ===========================
# tdcsfog model
# ===========================
tdcsfog_model_list1 = []
for i in tdcsfog_path1:
    model = TdcsfogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    tdcsfog_model_list1.append(model)

tdcsfog_model_list3_1 = []
for i in tdcsfog_path3_1:
    model = TdcsfogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    tdcsfog_model_list3_1.append(model)
    
tdcsfog_model_list3_2 = []
for i in tdcsfog_path3_2:
    model = TdcsfogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    tdcsfog_model_list3_2.append(model)
    
tdcsfog_model_list3_3 = []
for i in tdcsfog_path3_3:
    model = TdcsfogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    tdcsfog_model_list3_3.append(model)
    
tdcsfog_model_list4_1 = []
for i in tdcsfog_path4_1:
    model = TdcsfogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    tdcsfog_model_list4_1.append(model)
    
tdcsfog_model_list4_2 = []
for i in tdcsfog_path4_2:
    model = TdcsfogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    tdcsfog_model_list4_2.append(model)
    
tdcsfog_model_list4_3 = []
for i in tdcsfog_path4_3:
    model = TdcsfogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    tdcsfog_model_list4_3.append(model)
    

# ===========================
# defog model
# ===========================
defog_model_list2 = []
for i in defog_path2:
    model = DefogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    defog_model_list2.append(model)
    
defog_model_list4 = []
for i in defog_path4:
    model = DefogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    defog_model_list4.append(model)
    
defog_model_list5 = []
for i in defog_path5:
    model = DefogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    defog_model_list5.append(model)
    
defog_model_list6 = []
for i in defog_path6:
    model = DefogRnnModel2()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    defog_model_list6.append(model)

defog_model_list7 = []
for path in defog_path7:
    model = Defog3Model()
    state = torch.load(path, map_location=torch.device("cpu"))
    model.load_state_dict(state["model"])
    model = model.to(device)
    model.eval()
    defog_model_list7.append(model)
    print(f"load weights from {path}")

load weights from /kaggle/input/pd-exp238/fold0_best.pth
load weights from /kaggle/input/pd-exp238/fold1_best.pth
load weights from /kaggle/input/pd-exp238/fold2_best.pth
load weights from /kaggle/input/pd-exp238/fold3_best.pth
load weights from /kaggle/input/pd-exp238/fold4_best.pth


In [11]:
df_all = []

# tdcsfog

In [12]:
# =========================
# tdcsfog1
# =========================
th_len = 5000
w = 0.20

tdcsfog_list = glob.glob(TDCSFOG_DATA_PATH)
cols = ["AccV","AccML","AccAP"]
num_cols = ['AccV', 'AccML', 'AccAP', 
       'AccV_lag_diff', 'AccV_lead_diff', 'AccV_cumsum', 'AccML_lag_diff',
       'AccML_lead_diff', 'AccML_cumsum', 'AccAP_lag_diff', 'AccAP_lead_diff',
       'AccAP_cumsum']
for p in tqdm(tdcsfog_list):
    id_values = p.split("/")[-1].split(".")[0]
    df = pd.read_csv(p)
    
    if len(df) > th_len:
        seq_len = 5000
        shift = 2500
        offset = 1250
    else:
        seq_len = 3000
        shift = 1500
        offset = 750
        
    batch = (len(df)-1) // shift
    if batch == 0:
        batch = 1
    for c in cols:
        df[f"{c}_lag_diff"] = df[c].diff()
        df[f"{c}_lead_diff"] = df[c].diff(-1)
        df[f"{c}_cumsum"] = df[c].cumsum()
    sc = RobustScaler()
    df[num_cols] = sc.fit_transform(df[num_cols].values)
    df[num_cols] = df[num_cols].fillna(0)
    num = df[num_cols].values
    time = df["Time"].values
    
    num_array = np.zeros([batch,seq_len,12])
    mask_array = np.zeros([batch,seq_len],dtype=int)
    time_array = np.zeros([batch,seq_len],dtype=int)
    pred_use_array = np.zeros([batch,seq_len],dtype=int)
    
    if len(df) <= seq_len:
        b = 0
        num_ = num.copy()
        time_ = time.copy()
        num_len = len(num_)

        num_array[b,:num_len,:] = num_
        time_array[b,:num_len] = time_
        mask_array[b,:num_len] = 1
        pred_use_array[b,:num_len] = 1
    else:
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                time_ = time[b*shift : ]
                num_len = len(num_)

                num_array[b,:num_len,:] = num_
                time_array[b,:num_len] = time_
                mask_array[b,:num_len] = 1
                pred_use_array[b,offset:num_len] = 1
            elif b == 0:
                num_ = num[b*shift:b*shift+seq_len]
                time_ = time[b*shift:b*shift + seq_len]

                num_array[b,:,:] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,:shift+offset] = 1
            else:
                num_ = num[b*shift:b*shift+seq_len]
                time_ = time[b*shift:b*shift + seq_len]

                num_array[b,:,:] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,offset:shift+offset] = 1
            
            

    
    
    
    test_ = FogDataset(num_array,
                       mask_array,
                       train=False)
    test_loader = DataLoader(dataset=test_, 
                        batch_size=bs, 
                        shuffle = False)
    for n,m in enumerate(tdcsfog_model_list1):
        if n == 0:
            pred = make_pred(test_loader,m) / len(tdcsfog_model_list1)
        else:
            pred += make_pred(test_loader,m) / len(tdcsfog_model_list1)
    pred_list = []
    for i in range(batch):
        mask_ = pred_use_array[i]
        pred_ = pred[i,mask_ == 1,:]
        time_ = time_array[i, mask_ == 1]
        df_ = pd.DataFrame()
        df_["StartHesitation"] = pred_[:,0] * w
        df_["Turn"] = pred_[:,1] * w
        df_["Walking"] = pred_[:,2] * w
        df_["Time"] = time_
        df_["Id"] = id_values
        df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
        pred_list.append(df_)
    pred = pd.concat(pred_list).reset_index(drop=True)
    df_all.append(pred)

100%|██████████| 1/1 [00:01<00:00,  1.76s/it]

100%|██████████| 1/1 [00:00<00:00, 10.65it/s]

100%|██████████| 1/1 [00:00<00:00,  9.04it/s]

100%|██████████| 1/1 [00:00<00:00, 10.90it/s]

100%|██████████| 1/1 [00:02<00:00,  2.23s/it]


In [13]:
# =========================
# tdcsfog3
# =========================
th_len = 5000
w = 0.40

tdcsfog_list = glob.glob(TDCSFOG_DATA_PATH)
cols = ["AccV","AccML","AccAP"]
num_cols = ['AccV', 'AccML', 'AccAP', 
       'AccV_lag_diff', 'AccV_lead_diff', 'AccV_cumsum', 'AccML_lag_diff',
       'AccML_lead_diff', 'AccML_cumsum', 'AccAP_lag_diff', 'AccAP_lead_diff',
       'AccAP_cumsum']
for p in tqdm(tdcsfog_list):
    id_values = p.split("/")[-1].split(".")[0]
    df = pd.read_csv(p)
    if len(df) > th_len:
        seq_len = 5000
        shift = 2500
        offset = 1250
    else:
        seq_len = 3000
        shift = 1500
        offset = 750
    batch = (len(df)-1) // shift
    if batch == 0:
        batch = 1
    for c in cols:
        df[f"{c}_lag_diff"] = df[c].diff()
        df[f"{c}_lead_diff"] = df[c].diff(-1)
        df[f"{c}_cumsum"] = df[c].cumsum()
    sc = RobustScaler()
    df[num_cols] = sc.fit_transform(df[num_cols].values)
    df[num_cols] = df[num_cols].fillna(0)
    num = df[num_cols].values
    time = df["Time"].values
    
    num_array = np.zeros([batch,seq_len,12])
    mask_array = np.zeros([batch,seq_len],dtype=int)
    time_array = np.zeros([batch,seq_len],dtype=int)
    pred_use_array = np.zeros([batch,seq_len],dtype=int)
    
    if len(df) <= seq_len:
        b = 0
        num_ = num.copy()
        time_ = time.copy()
        num_len = len(num_)

        num_array[b,:num_len,:] = num_
        time_array[b,:num_len] = time_
        mask_array[b,:num_len] = 1
        pred_use_array[b,:num_len] = 1
    else:
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                time_ = time[b*shift : ]
                num_len = len(num_)

                num_array[b,:num_len,:] = num_
                time_array[b,:num_len] = time_
                mask_array[b,:num_len] = 1
                pred_use_array[b,offset:num_len] = 1
            elif b == 0:
                num_ = num[b*shift:b*shift+seq_len]
                time_ = time[b*shift:b*shift + seq_len]

                num_array[b,:,:] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,:shift+offset] = 1
            else:
                num_ = num[b*shift:b*shift+seq_len]
                time_ = time[b*shift:b*shift + seq_len]

                num_array[b,:,:] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,offset:shift+offset] = 1
    
    
    
    test_ = FogDataset(num_array,
                       mask_array,
                       train=False)
    test_loader = DataLoader(dataset=test_, 
                        batch_size=bs, 
                        shuffle = False)
    for n,m in enumerate(tdcsfog_model_list3_1):
        if n == 0:
            pred1 = make_pred(test_loader,m) / len(tdcsfog_model_list3_1)
        else:
            pred1 += make_pred(test_loader,m) / len(tdcsfog_model_list3_1)
    for n,m in enumerate(tdcsfog_model_list3_2):
        if n == 0:
            pred2 = make_pred(test_loader,m) / len(tdcsfog_model_list3_2)
        else:
            pred2 += make_pred(test_loader,m) / len(tdcsfog_model_list3_2)
    for n,m in enumerate(tdcsfog_model_list3_3):
        if n == 0:
            pred3 = make_pred(test_loader,m) / len(tdcsfog_model_list3_3)
        else:
            pred3 += make_pred(test_loader,m) / len(tdcsfog_model_list3_3)
    pred = pred1.copy()
    pred[:,:,1] = pred2[:,:,1]
    pred[:,:,2] = pred3[:,:,2]
    
    
    pred_list = []
    for i in range(batch):
        mask_ = pred_use_array[i]
        pred_ = pred[i,mask_ == 1,:]
        time_ = time_array[i, mask_ == 1]
        df_ = pd.DataFrame()
        df_["StartHesitation"] = pred_[:,0] * w
        df_["Turn"] = pred_[:,1] * w
        df_["Walking"] = pred_[:,2] * w
        df_["Time"] = time_
        df_["Id"] = id_values
        df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
        pred_list.append(df_)
    pred = pd.concat(pred_list).reset_index(drop=True)
    df_all.append(pred)

100%|██████████| 1/1 [00:00<00:00, 10.87it/s]

100%|██████████| 1/1 [00:00<00:00,  9.04it/s]

100%|██████████| 1/1 [00:00<00:00, 10.92it/s]

100%|██████████| 1/1 [00:00<00:00, 10.71it/s]

100%|██████████| 1/1 [00:00<00:00, 10.94it/s]

100%|██████████| 1/1 [00:00<00:00, 10.94it/s]

100%|██████████| 1/1 [00:00<00:00,  8.97it/s]

100%|██████████| 1/1 [00:00<00:00,  8.83it/s]

100%|██████████| 1/1 [00:00<00:00, 10.81it/s]

100%|██████████| 1/1 [00:00<00:00,  8.97it/s]

100%|██████████| 1/1 [00:00<00:00, 10.59it/s]

100%|██████████| 1/1 [00:00<00:00,  9.17it/s]

100%|██████████| 1/1 [00:00<00:00, 10.21it/s]

100%|██████████| 1/1 [00:00<00:00,  8.99it/s]

100%|██████████| 1/1 [00:01<00:00,  1.64s/it]


In [14]:
# =========================
# tdcsfog4
# =========================
th_len = 5000
w = 0.40

tdcsfog_list = glob.glob(TDCSFOG_DATA_PATH)
cols = ["AccV","AccML","AccAP"]
num_cols = ['AccV', 'AccML', 'AccAP', 
       'AccV_lag_diff', 'AccV_lead_diff', 'AccV_cumsum', 'AccML_lag_diff',
       'AccML_lead_diff', 'AccML_cumsum', 'AccAP_lag_diff', 'AccAP_lead_diff',
       'AccAP_cumsum']
for p in tqdm(tdcsfog_list):
    id_values = p.split("/")[-1].split(".")[0]
    df = pd.read_csv(p)
    if len(df) > th_len:
        seq_len = 5000
        shift = 2500
        offset = 1250
    else:
        seq_len = 3000
        shift = 1500
        offset = 750
    batch = (len(df)-1) // shift
    if batch == 0:
        batch = 1
    for c in cols:
        df[f"{c}_lag_diff"] = df[c].diff()
        df[f"{c}_lead_diff"] = df[c].diff(-1)
        df[f"{c}_cumsum"] = df[c].cumsum()
    sc = RobustScaler()
    df[num_cols] = sc.fit_transform(df[num_cols].values)
    df[num_cols] = df[num_cols].fillna(0)
    num = df[num_cols].values
    time = df["Time"].values
    
    num_array = np.zeros([batch,seq_len,12])
    mask_array = np.zeros([batch,seq_len],dtype=int)
    time_array = np.zeros([batch,seq_len],dtype=int)
    pred_use_array = np.zeros([batch,seq_len],dtype=int)
    
    if len(df) <= seq_len:
        b = 0
        num_ = num.copy()
        time_ = time.copy()
        num_len = len(num_)

        num_array[b,:num_len,:] = num_
        time_array[b,:num_len] = time_
        mask_array[b,:num_len] = 1
        pred_use_array[b,:num_len] = 1
    else:
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                time_ = time[b*shift : ]
                num_len = len(num_)

                num_array[b,:num_len,:] = num_
                time_array[b,:num_len] = time_
                mask_array[b,:num_len] = 1
                pred_use_array[b,offset:num_len] = 1
            elif b == 0:
                num_ = num[b*shift:b*shift+seq_len]
                time_ = time[b*shift:b*shift + seq_len]

                num_array[b,:,:] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,:shift+offset] = 1
            else:
                num_ = num[b*shift:b*shift+seq_len]
                time_ = time[b*shift:b*shift + seq_len]

                num_array[b,:,:] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,offset:shift+offset] = 1
    
    
    
    test_ = FogDataset(num_array,
                       mask_array,
                       train=False)
    test_loader = DataLoader(dataset=test_, 
                        batch_size=bs, 
                        shuffle = False)
    for n,m in enumerate(tdcsfog_model_list4_1):
        if n == 0:
            pred1 = make_pred(test_loader,m) / len(tdcsfog_model_list4_1)
        else:
            pred1 += make_pred(test_loader,m) / len(tdcsfog_model_list4_1)
    for n,m in enumerate(tdcsfog_model_list4_2):
        if n == 0:
            pred2 = make_pred(test_loader,m) / len(tdcsfog_model_list4_2)
        else:
            pred2 += make_pred(test_loader,m) / len(tdcsfog_model_list4_2)
    for n,m in enumerate(tdcsfog_model_list4_3):
        if n == 0:
            pred3 = make_pred(test_loader,m) / len(tdcsfog_model_list4_3)
        else:
            pred3 += make_pred(test_loader,m) / len(tdcsfog_model_list4_3)
    pred = pred1.copy()
    pred[:,:,0] = pred1[:,:,0]*0.5 + pred2[:,:,0]*0.5
    pred[:,:,1] = pred1[:,:,1]*0.5 + pred3[:,:,1]*0.5
    pred[:,:,2] = pred2[:,:,2]*0.5 + pred3[:,:,2]*0.5
    
    
    pred_list = []
    for i in range(batch):
        mask_ = pred_use_array[i]
        pred_ = pred[i,mask_ == 1,:]
        time_ = time_array[i, mask_ == 1]
        df_ = pd.DataFrame()
        df_["StartHesitation"] = pred_[:,0] * w
        df_["Turn"] = pred_[:,1] * w
        df_["Walking"] = pred_[:,2] * w
        df_["Time"] = time_
        df_["Id"] = id_values
        df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
        pred_list.append(df_)
    pred = pd.concat(pred_list).reset_index(drop=True)
    df_all.append(pred)

100%|██████████| 1/1 [00:00<00:00,  9.03it/s]

100%|██████████| 1/1 [00:00<00:00, 10.82it/s]

100%|██████████| 1/1 [00:00<00:00,  9.06it/s]

100%|██████████| 1/1 [00:00<00:00,  8.94it/s]

100%|██████████| 1/1 [00:00<00:00, 10.73it/s]

100%|██████████| 1/1 [00:00<00:00,  9.07it/s]

100%|██████████| 1/1 [00:00<00:00, 10.76it/s]

100%|██████████| 1/1 [00:00<00:00, 10.63it/s]

100%|██████████| 1/1 [00:00<00:00, 10.87it/s]

100%|██████████| 1/1 [00:00<00:00, 10.56it/s]

100%|██████████| 1/1 [00:00<00:00, 10.69it/s]

100%|██████████| 1/1 [00:00<00:00,  8.90it/s]

100%|██████████| 1/1 [00:00<00:00,  8.91it/s]

100%|██████████| 1/1 [00:00<00:00, 10.77it/s]

100%|██████████| 1/1 [00:01<00:00,  1.66s/it]


# defog

In [15]:
# =========================
# defog2
# =========================
th_len = 200000
w = 0.35

defog_list = glob.glob(DEFOG_DATA_PATH)
cols = ["AccV","AccML","AccAP"]
num_cols = ["AccV","AccML","AccAP",'AccV_lag_diff',
            'AccV_lead_diff', 'AccML_lag_diff', 'AccML_lead_diff',
            'AccAP_lag_diff', 'AccAP_lead_diff']
for p in tqdm(defog_list):
    id_values = p.split("/")[-1].split(".")[0]
    df = pd.read_csv(p)
    if len(df) > th_len:
        seq_len = 30000
        shift = 15000
        offset = 7500
    else:
        seq_len = 15000
        shift = 7500
        offset = 3750
    batch = (len(df)-1) // shift
    if batch == 0:
        batch = 1
    for c in cols:
        df[f"{c}_lag_diff"] = df[c].diff()
        df[f"{c}_lead_diff"] = df[c].diff(-1)
    sc = StandardScaler()
    df[num_cols] = sc.fit_transform(df[num_cols].values)
    df[num_cols] = df[num_cols].fillna(0)
    num = df[num_cols].values
    time = df["Time"].values
    
    num_array = np.zeros([batch,seq_len,9])
    mask_array = np.zeros([batch,seq_len],dtype=int)
    time_array = np.zeros([batch,seq_len],dtype=int)
    pred_use_array = np.zeros([batch,seq_len],dtype=int)
    
    if len(df) <= seq_len:
        b = 0
        num_len = len(num)
        num_array[b,:num_len,:] = num
        time_array[b,:num_len] = time
        mask_array[b,:num_len] = 1
        pred_use_array[b,:num_len] = 1
    else:
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                time_ = time[b*shift : ]
                num_len = len(num_)

                num_array[b,:num_len,:] = num_
                time_array[b,:num_len] = time_
                mask_array[b,:num_len] = 1
                pred_use_array[b,offset:num_len] = 1
            elif b == 0:
                num_ = num[b*shift:b*shift+seq_len]
                time_ = time[b*shift:b*shift + seq_len]

                num_array[b,:,:] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,:shift+offset] = 1
            else:
                num_ = num[b*shift:b*shift+seq_len]
                time_ = time[b*shift:b*shift + seq_len]

                num_array[b,:,:] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,offset:shift+offset] = 1  
    
    test_ = FogDataset(num_array,
                       mask_array,
                       train=False)
    test_loader = DataLoader(dataset=test_, 
                        batch_size=bs, 
                        shuffle = False)
    for n,m in enumerate(defog_model_list2):
        if n == 0:
            pred = make_pred(test_loader,m) / len(defog_model_list2)
        else:
            pred += make_pred(test_loader,m) / len(defog_model_list2)
    pred_list = []
    for i in range(batch):
        mask_ = pred_use_array[i]
        pred_ = pred[i,mask_ == 1,:]
        time_ = time_array[i, mask_ == 1]
        df_ = pd.DataFrame()
        df_["StartHesitation"] = pred_[:,0] * w
        df_["Turn"] = pred_[:,1] * w
        df_["Walking"] = pred_[:,2] * w
        df_["Time"] = time_
        df_["Id"] = id_values
        df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
        pred_list.append(df_)
    pred = pd.concat(pred_list).reset_index(drop=True)
    df_all.append(pred)

100%|██████████| 1/1 [00:01<00:00,  1.32s/it]

100%|██████████| 1/1 [00:01<00:00,  1.35s/it]

100%|██████████| 1/1 [00:01<00:00,  1.30s/it]

100%|██████████| 1/1 [00:01<00:00,  1.31s/it]

100%|██████████| 1/1 [00:07<00:00,  7.55s/it]


In [16]:
# =========================
# defog4
# =========================
th_len = 200000
w = 0.25

defog_list = glob.glob(DEFOG_DATA_PATH)
cols = ["AccV","AccML","AccAP"]
num_cols = ["AccV","AccML","AccAP",'AccV_lag_diff',
            'AccV_lead_diff', 'AccML_lag_diff', 'AccML_lead_diff',
            'AccAP_lag_diff', 'AccAP_lead_diff']
for p in tqdm(defog_list):
    id_values = p.split("/")[-1].split(".")[0]
    df = pd.read_csv(p)
    if len(df) > th_len:
        seq_len = 30000
        shift = 15000
        offset = 7500
    else:
        seq_len = 15000
        shift = 7500
        offset = 3750
    batch = (len(df)-1) // shift
    if batch == 0:
        batch = 1
    for c in cols:
        df[f"{c}_lag_diff"] = df[c].diff()
        df[f"{c}_lead_diff"] = df[c].diff(-1)
    sc = StandardScaler()
    df[num_cols] = sc.fit_transform(df[num_cols].values)
    df[num_cols] = df[num_cols].fillna(0)
    num = df[num_cols].values
    time = df["Time"].values
    
    num_array = np.zeros([batch,seq_len,9])
    mask_array = np.zeros([batch,seq_len],dtype=int)
    time_array = np.zeros([batch,seq_len],dtype=int)
    pred_use_array = np.zeros([batch,seq_len],dtype=int)
    
    if len(df) <= seq_len:
        b = 0
        num_len = len(num)
        num_array[b,:num_len,:] = num
        time_array[b,:num_len] = time
        mask_array[b,:num_len] = 1
        pred_use_array[b,:num_len] = 1
    else:
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                time_ = time[b*shift : ]
                num_len = len(num_)

                num_array[b,:num_len,:] = num_
                time_array[b,:num_len] = time_
                mask_array[b,:num_len] = 1
                pred_use_array[b,offset:num_len] = 1
            elif b == 0:
                num_ = num[b*shift:b*shift+seq_len]
                time_ = time[b*shift:b*shift + seq_len]

                num_array[b,:,:] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,:shift+offset] = 1
            else:
                num_ = num[b*shift:b*shift+seq_len]
                time_ = time[b*shift:b*shift + seq_len]

                num_array[b,:,:] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,offset:shift+offset] = 1  
    
    test_ = FogDataset(num_array,
                       mask_array,
                       train=False)
    test_loader = DataLoader(dataset=test_, 
                        batch_size=bs, 
                        shuffle = False)
    for n,m in enumerate(defog_model_list4):
        if n == 0:
            pred = make_pred(test_loader,m) / len(defog_model_list4)
        else:
            pred += make_pred(test_loader,m) / len(defog_model_list4)
    pred_list = []
    for i in range(batch):
        mask_ = pred_use_array[i]
        pred_ = pred[i,mask_ == 1,:]
        time_ = time_array[i, mask_ == 1]
        df_ = pd.DataFrame()
        df_["StartHesitation"] = pred_[:,0] * w
        df_["Turn"] = pred_[:,1] * w
        df_["Walking"] = pred_[:,2] * w
        df_["Time"] = time_
        df_["Id"] = id_values
        df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
        pred_list.append(df_)
    pred = pd.concat(pred_list).reset_index(drop=True)
    df_all.append(pred)

100%|██████████| 1/1 [00:01<00:00,  1.31s/it]

100%|██████████| 1/1 [00:01<00:00,  1.29s/it]

100%|██████████| 1/1 [00:01<00:00,  1.31s/it]

100%|██████████| 1/1 [00:01<00:00,  1.29s/it]

100%|██████████| 1/1 [00:07<00:00,  7.31s/it]


In [17]:
# =========================
# defog5
# =========================
th_len = 200000
w = 0.25

defog_list = glob.glob(DEFOG_DATA_PATH)
cols = ["AccV","AccML","AccAP"]
num_cols = ["AccV","AccML","AccAP",'AccV_lag_diff',
            'AccV_lead_diff', 'AccML_lag_diff', 'AccML_lead_diff',
            'AccAP_lag_diff', 'AccAP_lead_diff']
for p in tqdm(defog_list):
    id_values = p.split("/")[-1].split(".")[0]
    df = pd.read_csv(p)
    if len(df) > th_len:
        seq_len = 30000
        shift = 15000
        offset = 7500
    else:
        seq_len = 15000
        shift = 7500
        offset = 3750
    batch = (len(df)-1) // shift
    if batch == 0:
        batch = 1
    for c in cols:
        df[f"{c}_lag_diff"] = df[c].diff()
        df[f"{c}_lead_diff"] = df[c].diff(-1)
    sc = StandardScaler()
    df[num_cols] = sc.fit_transform(df[num_cols].values)
    df[num_cols] = df[num_cols].fillna(0)
    num = df[num_cols].values
    time = df["Time"].values
    
    num_array = np.zeros([batch,seq_len,9])
    mask_array = np.zeros([batch,seq_len],dtype=int)
    time_array = np.zeros([batch,seq_len],dtype=int)
    pred_use_array = np.zeros([batch,seq_len],dtype=int)
    
    if len(df) <= seq_len:
        b = 0
        num_len = len(num)
        num_array[b,:num_len,:] = num
        time_array[b,:num_len] = time
        mask_array[b,:num_len] = 1
        pred_use_array[b,:num_len] = 1
    else:
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                time_ = time[b*shift : ]
                num_len = len(num_)

                num_array[b,:num_len,:] = num_
                time_array[b,:num_len] = time_
                mask_array[b,:num_len] = 1
                pred_use_array[b,offset:num_len] = 1
            elif b == 0:
                num_ = num[b*shift:b*shift+seq_len]
                time_ = time[b*shift:b*shift + seq_len]

                num_array[b,:,:] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,:shift+offset] = 1
            else:
                num_ = num[b*shift:b*shift+seq_len]
                time_ = time[b*shift:b*shift + seq_len]

                num_array[b,:,:] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,offset:shift+offset] = 1  
    
    test_ = FogDataset(num_array,
                       mask_array,
                       train=False)
    test_loader = DataLoader(dataset=test_, 
                        batch_size=bs, 
                        shuffle = False)
    for n,m in enumerate(defog_model_list5):
        if n == 0:
            pred = make_pred(test_loader,m) / len(defog_model_list5)
        else:
            pred += make_pred(test_loader,m) / len(defog_model_list5)
    pred_list = []
    for i in range(batch):
        mask_ = pred_use_array[i]
        pred_ = pred[i,mask_ == 1,:]
        time_ = time_array[i, mask_ == 1]
        df_ = pd.DataFrame()
        df_["StartHesitation"] = pred_[:,0] * w
        df_["Turn"] = pred_[:,1] * w
        df_["Walking"] = pred_[:,2] * w
        df_["Time"] = time_
        df_["Id"] = id_values
        df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
        pred_list.append(df_)
    pred = pd.concat(pred_list).reset_index(drop=True)
    df_all.append(pred)

100%|██████████| 1/1 [00:01<00:00,  1.29s/it]

100%|██████████| 1/1 [00:01<00:00,  1.33s/it]

100%|██████████| 1/1 [00:01<00:00,  1.45s/it]

100%|██████████| 1/1 [00:01<00:00,  1.30s/it]

100%|██████████| 1/1 [00:07<00:00,  7.43s/it]


In [18]:
# =========================
# defog6
# =========================
th_len = 200000
w = 0.10

defog_list = glob.glob(DEFOG_DATA_PATH)
cols = ["AccV","AccML","AccAP"]
num_cols = ["AccV","AccML","AccAP",'AccV_lag_diff',
            'AccV_lead_diff', 'AccML_lag_diff', 'AccML_lead_diff',
            'AccAP_lag_diff', 'AccAP_lead_diff']
for p in tqdm(defog_list):
    id_values = p.split("/")[-1].split(".")[0]
    df = pd.read_csv(p)
    if len(df) > th_len:
        seq_len = 30000
        shift = 15000
        offset = 7500
    else:
        seq_len = 15000
        shift = 7500
        offset = 3750
    batch = (len(df)-1) // shift
    if batch == 0:
        batch = 1
    for c in cols:
        df[f"{c}_lag_diff"] = df[c].diff()
        df[f"{c}_lead_diff"] = df[c].diff(-1)
    sc = StandardScaler()
    df[num_cols] = sc.fit_transform(df[num_cols].values)
    df[num_cols] = df[num_cols].fillna(0)
    num = df[num_cols].values
    time = df["Time"].values
    
    num_array = np.zeros([batch,seq_len,9])
    mask_array = np.zeros([batch,seq_len],dtype=int)
    time_array = np.zeros([batch,seq_len],dtype=int)
    pred_use_array = np.zeros([batch,seq_len],dtype=int)
    
    if len(df) <= seq_len:
        b = 0
        num_len = len(num)
        num_array[b,:num_len,:] = num
        time_array[b,:num_len] = time
        mask_array[b,:num_len] = 1
        pred_use_array[b,:num_len] = 1
    else:
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                time_ = time[b*shift : ]
                num_len = len(num_)

                num_array[b,:num_len,:] = num_
                time_array[b,:num_len] = time_
                mask_array[b,:num_len] = 1
                pred_use_array[b,offset:num_len] = 1
            elif b == 0:
                num_ = num[b*shift:b*shift+seq_len]
                time_ = time[b*shift:b*shift + seq_len]

                num_array[b,:,:] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,:shift+offset] = 1
            else:
                num_ = num[b*shift:b*shift+seq_len]
                time_ = time[b*shift:b*shift + seq_len]

                num_array[b,:,:] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,offset:shift+offset] = 1  
    
    test_ = FogDataset(num_array,
                       mask_array,
                       train=False)
    test_loader = DataLoader(dataset=test_, 
                        batch_size=bs, 
                        shuffle = False)
    for n,m in enumerate(defog_model_list6):
        if n == 0:
            pred = make_pred(test_loader,m) / len(defog_model_list6)
        else:
            pred += make_pred(test_loader,m) / len(defog_model_list6)
    pred_list = []
    for i in range(batch):
        mask_ = pred_use_array[i]
        pred_ = pred[i,mask_ == 1,:]
        time_ = time_array[i, mask_ == 1]
        df_ = pd.DataFrame()
        df_["StartHesitation"] = pred_[:,0] * w
        df_["Turn"] = pred_[:,1] * w
        df_["Walking"] = pred_[:,2] * w
        df_["Time"] = time_
        df_["Id"] = id_values
        df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
        pred_list.append(df_)
    pred = pd.concat(pred_list).reset_index(drop=True)
    df_all.append(pred)

100%|██████████| 1/1 [00:01<00:00,  1.91s/it]

100%|██████████| 1/1 [00:01<00:00,  1.91s/it]

100%|██████████| 1/1 [00:01<00:00,  1.91s/it]

100%|██████████| 1/1 [00:01<00:00,  1.91s/it]

100%|██████████| 1/1 [00:10<00:00, 10.30s/it]


In [19]:
# =========================
# defog7
# =========================
th_len = 200000
w = 0.05

defog_list = glob.glob(DEFOG_DATA_PATH)
cols = ["AccV","AccML","AccAP"]
num_cols = ["AccV","AccML","AccAP",'AccV_lag_diff',
            'AccV_lead_diff', 'AccML_lag_diff', 'AccML_lead_diff',
            'AccAP_lag_diff', 'AccAP_lead_diff']
for p in tqdm(defog_list):
    id_values = p.split("/")[-1].split(".")[0]
    df = pd.read_csv(p)
    if len(df) > th_len:
        seq_len = 30000
        shift = 15000
        offset = 7500
    else:
        seq_len = 15000
        shift = 7500
        offset = 3750
    batch = (len(df)-1) // shift
    if batch == 0:
        batch = 1
    for c in cols:
        df[f"{c}_lag_diff"] = df[c].diff()
        df[f"{c}_lead_diff"] = df[c].diff(-1)
    sc = StandardScaler()
    df[num_cols] = sc.fit_transform(df[num_cols].values)
    df[num_cols] = df[num_cols].fillna(0)
    num = df[num_cols].values
    time = df["Time"].values
    
    num_array = np.zeros([batch,seq_len,9])
    mask_array = np.zeros([batch,seq_len],dtype=int)
    time_array = np.zeros([batch,seq_len],dtype=int)
    pred_use_array = np.zeros([batch,seq_len],dtype=int)
    
    if len(df) <= seq_len:
        b = 0
        num_len = len(num)
        num_array[b,:num_len,:] = num
        time_array[b,:num_len] = time
        mask_array[b,:num_len] = 1
        pred_use_array[b,:num_len] = 1
    else:
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                time_ = time[b*shift : ]
                num_len = len(num_)

                num_array[b,:num_len,:] = num_
                time_array[b,:num_len] = time_
                mask_array[b,:num_len] = 1
                pred_use_array[b,offset:num_len] = 1
            elif b == 0:
                num_ = num[b*shift:b*shift+seq_len]
                time_ = time[b*shift:b*shift + seq_len]

                num_array[b,:,:] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,:shift+offset] = 1
            else:
                num_ = num[b*shift:b*shift+seq_len]
                time_ = time[b*shift:b*shift + seq_len]

                num_array[b,:,:] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,offset:shift+offset] = 1  
    
    test_ = FogDataset(num_array,
                       mask_array,
                       train=False)
    test_loader = DataLoader(dataset=test_, 
                        batch_size=bs, 
                        shuffle = False)
    for n,m in enumerate(defog_model_list7):
        if n == 0:
            pred = make_pred(test_loader,m) / len(defog_model_list7)
        else:
            pred += make_pred(test_loader,m) / len(defog_model_list7)
    pred_list = []
    for i in range(batch):
        mask_ = pred_use_array[i]
        pred_ = pred[i,mask_ == 1,:]
        time_ = time_array[i, mask_ == 1]
        df_ = pd.DataFrame()
        df_["StartHesitation"] = pred_[:,0] * w
        df_["Turn"] = pred_[:,1] * w
        df_["Walking"] = pred_[:,2] * w
        df_["Time"] = time_
        df_["Id"] = id_values
        df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
        pred_list.append(df_)
    pred = pd.concat(pred_list).reset_index(drop=True)
    df_all.append(pred)

100%|██████████| 1/1 [00:01<00:00,  1.29s/it]

100%|██████████| 1/1 [00:01<00:00,  1.34s/it]

100%|██████████| 1/1 [00:01<00:00,  1.29s/it]

100%|██████████| 1/1 [00:01<00:00,  1.30s/it]

100%|██████████| 1/1 [00:07<00:00,  7.29s/it]


In [20]:
df_all = pd.concat(df_all).reset_index(drop=True)
df_all = df_all.groupby(by="Id")[['StartHesitation', 'Turn', 'Walking']].sum().reset_index()
df_all[['Id', 'StartHesitation', 'Turn', 'Walking']].to_csv("submission.csv",index=False)

In [21]:
df_all

,Id,StartHesitation,Turn,Walking
0,003f117e14_0,0.002737,0.003436,0.000623
1,003f117e14_1,0.002624,0.003394,0.000576
2,003f117e14_10,0.002088,0.002961,0.000458
3,003f117e14_100,0.001648,0.002645,0.000372
4,003f117e14_1000,0.002842,0.037812,0.000685
...,...,...,...,...
286365,02ab235146_99995,0.000296,0.277196,0.036089
286366,02ab235146_99996,0.000290,0.275381,0.035312
286367,02ab235146_99997,0.000286,0.275587,0.034789
286368,02ab235146_99998,0.000284,0.278020,0.034215
